# Importation des bibliothèques nécessaires

In [1]:
import pandas as pd
from pandas import ExcelWriter
import xlsxwriter
from pandas import ExcelFile
from datetime import datetime
import datetime
from datetime import timedelta
import numpy as np
import os
import statistics
from dateutil.parser import parse
import math
import calendar
from sklearn.cluster import DBSCAN
import joblib

In [2]:
# Charger le modèle à partir du fichier picklé
loaded_model = joblib.load("svm_model.pkl")

# Création de deux fonctions utilisables dans l'importation des données

In [3]:

def toZero(x):
    try:
        converted_value = int(x)
    except ValueError:
        converted_value = 0
    return converted_value

def findDay(date):
    day, month, year = (int(i) for i in date.split('/'))
    dayNumber = calendar.weekday(year, month, day)
    return dayNumber

def arrondir_multiple_de_5(nombre):
    multiple_de_5 = round(nombre / 5) * 5
    return multiple_de_5

def process_data(df_Fusion, seuil):
    groupe = 0
    df_Fusion['Groupe'] = 0

    for index, row in df_Fusion.iterrows():
        if index == 0:
            df_Fusion.at[index, 'Groupe'] = groupe
        else:
            diff = abs(row['Talon surconso identifie'] - df_Fusion.at[index - 1, 'Talon surconso identifie'])
            if diff <= seuil:
                df_Fusion.at[index, 'Groupe'] = groupe
            else:
                groupe += 1
                df_Fusion.at[index, 'Groupe'] = groupe

    df_simplified = df_Fusion.groupby(['Groupe', 'Début surconsommation', 'Fin surconsommation']).agg({
        'Code': 'first',  # Include the 'Code' column from the original dataframe
        'Energie': 'first',  # Include the 'Energie' column from the original dataframe
        'heure ouverture': 'first',  # Include the 'heure ouverture' column from the original dataframe
        'heure fermetur': 'first',  # Include the 'heure fermetur' column from the original dataframe
        'TalonRef': 'first',  # Include the 'TalonRef' column from the original dataframe
        'Début surconsommation': 'first',
        'Fin surconsommation': 'first',
        'Talon surconso identifie': 'mean',
        'impact': 'first',  # Include the 'impact' column from the original dataframe
        'NbrHeures': 'sum',
        'NbrNuits': 'first',  # Include the 'NbrNuits' column from the original dataframe
        'Impact conso (kWh)': 'first',  # Include the 'Impact conso (kWh)' column from the original dataframe
        '% Surconso': 'first',  # Include the '% Surconso' column from the original dataframe
        "Période d'alerte": 'first',  # Include the "Période d'alerte" column from the original dataframe
        'Pourcentage de précision': 'first'  # Include the 'Pourcentage de précision' column from the original dataframe
    }).reset_index(drop=True)

    return df_simplified

# Création de l'algorithme de machine learning personnalisé que nous allons utiliser dans la fusion des heures de surconsommation.

# Cet algorithme est un algorithme de clustering non supervisé.

# Il prend en compte la successivité des heures et une marge de surconsommation.

In [4]:
import numpy as np
from sklearn.base import BaseEstimator, ClusterMixin

class TimeMarginClustering(BaseEstimator, ClusterMixin):
    def __init__(self, time_margin=1, value_margin=20):
        self.time_margin = time_margin
        self.value_margin = value_margin
        self.labels_ = None

    def fit(self, X):
        # Sort the data based on the time column (column 0)
        sorted_indices = np.argsort(X[:, 0])
        sorted_X = X[sorted_indices]

        # Initialize variables for clustering
        cluster_labels = np.zeros(len(X), dtype=int)
        current_cluster = 0
        prev_time = sorted_X[0, 0]
        prev_value = sorted_X[0, 1]

        # Iterate over the sorted data and assign cluster labels
        for i in range(len(sorted_X)):
            time = sorted_X[i, 0]
            value = sorted_X[i, 1]

            # Check if the time difference exceeds the time margin
            time_diff = time - prev_time
            if time_diff > self.time_margin:
                current_cluster += 1

            # Check if the value difference exceeds the value margin
            value_diff = abs(value - prev_value)
            if time_diff <= self.time_margin and value_diff > self.value_margin:
                current_cluster += 1

            # Assign the cluster label
            cluster_labels[i] = current_cluster

            # Update previous time and value
            prev_time = time
            prev_value = value

        # Assign the cluster labels to the algorithm's attribute
        self.labels_ = cluster_labels
        return self

    def fit_predict(self, X):
        self.fit(X)
        return self.labels_

    def predict(self, X):
        return self.fit_predict(X)

# Importation des données.

# Prétraitement des données (nettoyage des données)

# La détection de la surconsommation pour les nuits.

# L'appel et l'application de l'algorithme de fusion des heures pour les nuits.

In [5]:
writer = pd.ExcelWriter('suivi_9_sites.xlsx', engine='xlsxwriter')

Code_list=[]
TalonRef_list=[]
Date1_list=[]
Date2_list=[]
heurev_list=[]
heuref_list=[]
NbrHeure_list=[]
defrence_list=[]
surconso_identifie=[]
Energie=[]
Marge_list=[]



# Charger les données du fichier CSV
df = pd.read_csv('data.csv', sep=',', parse_dates=True, dtype={"Date":"str","Time":"str","salesforceName":"str","siteName":"str","Real Consumption (kWh)":"int64" , "TalonRef":"int64" , "Heure Ouverture":"int64" , "Heure Fermeture":"int64" , "Ouverture dim":"str" , "Margin":"int64"},converters={'Real Consumption (kWh)':toZero},encoding="utf-8-sig")
# Filtrer les données pour l'entreprise spécifiée
#print(df.columns)
for code in df["salesforceName"].unique():
    df2 = df[df["salesforceName"] == code]

    # Sélectionner les colonnes pertinentes (Date, Time, Real Consumption (kWh) , talont , ouverture , fermeture)
    df3 = df2[['Date', 'Time', 'Real Consumption (kWh)',"TalonRef" , "Heure Ouverture" , "Heure Fermeture" , "Margin"]]

    # Convertir la colonne 'Date' en type datetime avec le format approprié
    df3['Date'] = pd.to_datetime(df3['Date'], format='%Y-%m-%d')

    # Convertir la colonne 'Time' en type timedelta
    df3['Time'] = pd.to_datetime(df3['Time'], format='%H:%M:%S')

    # Combiner les colonnes 'Date' et 'Time' en une seule colonne de type datetime
    df3['DateTime'] = df3['Date'] + pd.to_timedelta(df3['Time'].dt.strftime('%H:%M:%S'))

    # Définition des heures d'ouverture et de fermeture de l'entreprise
    heure_ouverture = df3["Heure Ouverture"].iloc[0]
    heure_fermeture = df3["Heure Fermeture"].iloc[0]+1

    # Définition du talon de consommation
    talon_consommation = df3["TalonRef"].iloc[0]
    marge=df3["Margin"].iloc[0]

    # Filtrer les données pour les heures en dehors des heures d'ouverture de l'entreprise
    if heure_fermeture == 0:
        donnees_filtrees = df3[df3['DateTime'].dt.hour < heure_ouverture]
    else:
        donnees_filtrees = df3[(df3['DateTime'].dt.hour >= heure_fermeture) | (df3['DateTime'].dt.hour < heure_ouverture)]


    # Calculer la différence entre la consommation réelle et le talon de consommation
    donnees_filtrees['Difference'] = donnees_filtrees['Real Consumption (kWh)'] - talon_consommation

    # Filtrer les heures de surconsommation (différence > talon_consommation * 0.08)
    heures_surconsommation = donnees_filtrees[donnees_filtrees['Difference'] > talon_consommation * 0.08]

    print("\n")
    print("\n")
    print("l'analyse de site : {}", code)
    print("\n")
    print("\n")

    # Liste des dates uniques dans votre DataFrame
    dates_uniques = heures_surconsommation['Date'].unique()

    # Parcourir chaque date et effectuer le clustering
    for i in range(len(dates_uniques) - 1):
        date_actuelle = dates_uniques[i]
        date_suivante = dates_uniques[i + 1]

        # Filtrer les données pour la date actuelle et la date suivante
        heures_nuit = heures_surconsommation[
            (heures_surconsommation['Date'] == date_actuelle) &
            (heures_surconsommation['DateTime'].dt.hour >= heure_fermeture)
        ]

        heures_nuit_suivante = heures_surconsommation[
            (heures_surconsommation['Date'] == date_suivante) &
            (heures_surconsommation['DateTime'].dt.hour < heure_ouverture)
        ]


        # Combiner les données de la nuit actuelle et de la nuit suivante
        if heure_fermeture == 0:
            heures_date = heures_nuit_suivante
        else:
            heures_date = pd.concat([heures_nuit, heures_nuit_suivante])

        #print(heures_date.head(10))

        # Convertir les colonnes de surconsommation et d'impact en tableaux Numpy
        surconsommation = heures_date['DateTime'].values.reshape(-1, 1)
        # Extraire les indices et les valeurs de la colonne "Difference" des données heures_date
        indices = heures_date.index
        differences = heures_date['Difference'].values

        # Créer le tableau NumPy à partir des indices et des valeurs
        impact = np.column_stack((indices, differences))

        print(impact)
        
        if impact.size == 0:
             continue
        else:
            # Créer l'objet DBSCAN
            clustering_algo = TimeMarginClustering(time_margin=2, value_margin=marge)


            labels = clustering_algo.fit_predict(impact)

            # Ajouter les labels de clustering comme une nouvelle colonne
            heures_date['Cluster'] = labels

            # Calculer la valeur moyenne de l'impact pour chaque cluster
            clusters_moyenne = heures_date.groupby('Cluster')['Difference'].mean().reset_index()

            # Compter le nombre d'heures regroupées dans chaque cluster
            clusters_compte = heures_date.groupby('Cluster')['DateTime'].count().reset_index()
            clusters_compte = clusters_compte.rename(columns={'DateTime': 'Nombre d\'heures regroupées'})

            # Fusionner les informations de valeur moyenne et de compte dans un seul DataFrame
            clusters_info = clusters_moyenne.merge(clusters_compte, on='Cluster')

            #saver dans un excel
            for index, row in clusters_info.iterrows():
                heuref_list.append(heure_fermeture - 1)
                Code_list.append(code)
                Energie.append('Electricité')
                TalonRef_list.append(talon_consommation)
                date1 = dates_uniques[i].to_pydatetime().date()
                #date1 = datetime.datetime.strptime(date1, '%Y-%m-%d').date()
                Date1_list.append(date1)
                date2 = dates_uniques[i + 1].to_pydatetime().date()
                #date2 = datetime.datetime.strptime(date2, '%Y-%m-%d').date()
                Date2_list.append(date2)
                heurev_list.append(heure_ouverture)
                NbrHeure_list.append(row['Nombre d\'heures regroupées'])
                defrence_list.append(int(row['Difference']))
                surconso_identifie.append(talon_consommation+int(row['Difference']))
                Marge_list.append(marge)

            # Afficher les informations des clusters pour la date spécifique
            print("\n")
            print(f"Date: {dates_uniques[i]}")
            print("\n")
            print(clusters_info)
            print("\n")
          

df_NuitOut = pd.DataFrame({'Code':Code_list, 'Energie': Energie,'heure ouverture':heurev_list,'heure fermetur':heuref_list, 'TalonRef':TalonRef_list, 'Début surconsommation':Date1_list, 'Fin surconsommation':Date2_list, 'Talon surconso identifie':surconso_identifie, 'impact':defrence_list,'NbrHeures':NbrHeure_list, 'Marge':Marge_list})

C:\Users\Lahoucine\anaconda3\lib\site-packages\pandas\util\_decorators.py:311: ParserWarning: Both a converter and dtype were specified for column Real Consumption (kWh) - only the converter will be used.
  return func(*args, **kwargs)
C:\Users\Lahoucine\AppData\Local\Temp\ipykernel_13368\3107651127.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['Date'] = pd.to_datetime(df3['Date'], format='%Y-%m-%d')
C:\Users\Lahoucine\AppData\Local\Temp\ipykernel_13368\3107651127.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-





l'analyse de site : {} MO0157




[[23 64]
 [24 62]
 [25 62]
 [26 55]
 [27 55]
 [28 60]]


AttributeError: 'numpy.datetime64' object has no attribute 'to_pydatetime'

# La détection de la surconsommation pour les dimanches.

# L'appel et l'application de l'algorithme de fusion des heures pour les dimanches.

# L'organisation des sorties dans une feuille d'un fichier Excel.

In [ ]:
dCode_list=[]
dTalonRef_list=[]
dDate1_list=[]
dDate2_list=[]
dheurev_list=[]
dheuref_list=[]
dNbrHeure_list=[]
ddefrence_list=[]
dsurconso_identifie=[]
dEnergie=[]
dNbrNuit=[]
Impact_conso = []
perSurconso = []
Periode = []


# Convert 'Date' column to datetime type if needed
df['Date'] = pd.to_datetime(df['Date'])

# Filter the data to keep only Sundays
sundays_data = df[df['Date'].dt.weekday == 6]

# Print the filtered data
#print(sundays_data.head(30))
df = sundays_data
for code in df["salesforceName"].unique():
    df2 = df[df["salesforceName"] == code]

    # Select relevant columns: Date, Time, Real Consumption (kWh), TalonRef, Heure Ouverture, Heure Fermeture, Ouverture dim
    df3 = df2[['Date', 'Time', 'Real Consumption (kWh)', 'TalonRef', 'Heure Ouverture', 'Heure Fermeture', 'Ouverture dim' , 'Margin']]

    # Convert the 'Date' column to datetime type with the appropriate format
    df3['Date'] = pd.to_datetime(df3['Date'], format='%d/%m/%Y')

    # Convert the 'Time' column to timedelta type
    df3['Time'] = pd.to_timedelta(df3['Time'])

    # Combine the 'Date' and 'Time' columns into a single datetime column
    df3['DateTime'] = df3['Date'] + df3['Time']

    # Define the opening and closing hours of the company
    heure_ouverture = df3["Heure Ouverture"].iloc[0]
    heure_fermeture = df3["Heure Fermeture"].iloc[0] + 1
    ouverture_dim = df3["Ouverture dim"].iloc[0]

    # Define the consumption threshold
    talon_consommation = df3["TalonRef"].iloc[0]
    marge=df3["Margin"].iloc[0]

    # Filter the data for hours outside the company's opening hours
    if ouverture_dim == "Journée":
        donnees_filtrees = pd.DataFrame()
    elif ouverture_dim == "Fermé":
        donnees_filtrees = df3[(df3['DateTime'].dt.hour >= heure_ouverture) & (df3['DateTime'].dt.hour < heure_fermeture)]
    elif ouverture_dim == "Matin":
        donnees_filtrees = df3[(df3['DateTime'].dt.hour >= 14) & (df3['DateTime'].dt.hour < heure_fermeture)]

    if len(donnees_filtrees) > 0:
        # Calculate the difference between the actual consumption and the consumption threshold
        donnees_filtrees['Difference'] = donnees_filtrees['Real Consumption (kWh)'] - talon_consommation

        # Filter the overconsumption hours (difference > talon_consommation * 0.08)
        heures_surconsommation = donnees_filtrees[donnees_filtrees['Difference'] > talon_consommation * 0.08]

        # Print the hours of overconsumption
        if len(heures_surconsommation) > 0:
            print(heures_surconsommation)

        print("\n")
        print("\n")
        print("l'analyse de site : {}", code)
        print("\n")
        print("\n")

        print(len(heures_surconsommation))

        # Liste des dates uniques dans votre DataFrame
        dates_uniques = heures_surconsommation['Date'].unique()
        
        # Parcourir chaque date et effectuer le clustering
        for i in range(len(dates_uniques)-1):
            date = dates_uniques[i+1]

            heures_date = heures_surconsommation[(heures_surconsommation['Date'] == date)]
            #print(heures_date.head(10))

            # Convertir les colonnes de surconsommation et d'impact en tableaux Numpy
            surconsommation = heures_date['DateTime'].values.reshape(-1, 1)
            # Extraire les indices et les valeurs de la colonne "Difference" des données heures_date
            indices = heures_date.index
            differences = heures_date['Difference'].values

            # Créer le tableau NumPy à partir des indices et des valeurs
            impact = np.column_stack((indices, differences))

            
            # Créer l'objet DBSCAN
            clustering_algo = TimeMarginClustering(time_margin=2, value_margin=marge)

            labels = clustering_algo.fit_predict(impact)

            # Ajouter les labels de clustering comme une nouvelle colonne
            heures_date['Cluster'] = labels

            # Calculer la valeur moyenne de l'impact pour chaque cluster
            clusters_moyenne = heures_date.groupby('Cluster')['Difference'].mean().reset_index()

            # Compter le nombre d'heures regroupées dans chaque cluster
            clusters_compte = heures_date.groupby('Cluster')['DateTime'].count().reset_index()
            clusters_compte = clusters_compte.rename(columns={'DateTime': 'Nombre d\'heures regroupées'})

            # Fusionner les informations de valeur moyenne et de compte dans un seul DataFrame
            clusters_info = clusters_moyenne.merge(clusters_compte, on='Cluster')

            #saver dans un excel
            for index, row in clusters_info.iterrows():
                dheuref_list.append(heure_fermeture - 1)
                dCode_list.append(code)
                dEnergie.append('Electricité')
                dTalonRef_list.append(talon_consommation)
                date1 = dates_uniques[i+1].to_pydatetime().date()
                dDate1_list.append(date1)
               # date_string = np.datetime_as_string(dates_uniques[i+1], unit='D')
                #date2 = datetime.datetime.strptime(date_string, '%Y-%m-%d').date()
                dDate2_list.append(date1)
                dheurev_list.append(heure_ouverture)
                dNbrHeure_list.append(row['Nombre d\'heures regroupées'])
                ddefrence_list.append(int(row['Difference']))
                dsurconso_identifie.append(arrondir_multiple_de_5(talon_consommation+int(row['Difference'])))
                surconso = row['Nombre d\'heures regroupées'] * int(row['Difference'])
                dNbrNuit.append(1)
                Impact_conso.append(surconso)

                persurconso = (int(row['Difference']) /  talon_consommation)*100
                persurconso = int(persurconso)
                persurconso = str(persurconso)
                persurconso = persurconso + '%'
                perSurconso.append(persurconso)
                Periode.append('Dim')

df_dimOut = pd.DataFrame({'Code':dCode_list, 'Energie': dEnergie,'heure ouverture':dheurev_list,'heure fermetur':dheuref_list, 'TalonRef':dTalonRef_list, 'Début surconsommation':dDate1_list, 'Fin surconsommation':dDate2_list, 'Talon surconso identifie':dsurconso_identifie, 'impact':ddefrence_list,'NbrHeures':dNbrHeure_list, 'NbrNuits':dNbrNuit, 'Impact conso (kWh)':Impact_conso, '% Surconso':perSurconso, "Période d'alerte":Periode})
grouped_df = df_dimOut.groupby("Code").size().reset_index(name="Nombre de lignes")
grouped_df["Pourcentage de précision"] = 100 - (grouped_df["Nombre de lignes"] - 1) * 10
df = df_dimOut.merge(grouped_df[["Code", "Pourcentage de précision"]], on="Code", how="left")
df['Pourcentage de précision'] = df['Pourcentage de précision'].astype(str) + '%'
df = process_data(df , 5)

# Prétraitement des données et appel du modèle de machine learning (Random Forest) pour dimanches.

In [ ]:
data = df[["Talon surconso identifie", "impact", "NbrHeures", "NbrNuits", "Impact conso (kWh)", "% Surconso", "Période d'alerte"]]
data.head()

In [ ]:
print(data.dtypes)

In [ ]:
# Transformation de la colonne '% Surconso' en float
data['% Surconso'] = data['% Surconso'].apply(lambda x: float(x.strip('%')) / 100)

# Affichage du dataframe après la transformation
data.head()

In [ ]:
# Créer un dictionnaire de mapping
mapping = {"Nuit": 0, "Dim": 1, "Jour": 2, "Jour/nuit": 3, "Jour/Nuit": 3}

# Remplacer les valeurs de la colonne 'Catégorie' par les entiers correspondants
data["Période d'alerte"] = data["Période d'alerte"].replace(mapping)

# Afficher les 5 premières lignes du DataFrame
data.head()

In [ ]:
# Faire des prédictions pour chaque ligne de la dataframe
predictions = loaded_model.predict(data)
df["Catégorie"] = predictions
# Créer un dictionnaire de mapping
mapping = {0: "Investigation en cours", 1: "Changement de comportement", 2: "Eclairage", 3: "Froid", 4: "CVC", 5:"Multi-usage"}

# Remplacer les valeurs de la colonne 'Catégorie' par les entiers correspondants
df['Catégorie'] = df['Catégorie'].replace(mapping)
df.head(15)

In [ ]:
df.to_excel(writer, sheet_name='dim', index=False)

# Création de l'algorithme de machine learning personnalisé que nous allons utiliser dans la fusion des nuits de surconsommation.

# Il prend en compte la successivité des dates, une marge de surconsommation et une marge des heures.

In [ ]:
def custom_clustering(X, date_margin, consumption_margin, hours_margin, cond):
    clusters = []
    remaining_points = X.copy()

    while len(remaining_points) > 0:
        current_point = remaining_points[0]
        cluster = [current_point]
        remaining_points = np.delete(remaining_points, 0, axis=0)
        i = 0

        # Sauvegarde de la valeur originale de date_margin
        original_date_margin = date_margin

        while i < len(remaining_points):
            point = remaining_points[i]


            if (abs(point[0] - current_point[0]) <= date_margin and
                abs(point[1] - current_point[1]) <= consumption_margin and
                abs(point[2] - current_point[2]) <= hours_margin):

                current_point = point
                cluster.append(point)
                remaining_points = np.delete(remaining_points, i, axis=0)
            else:
                i += 1

            # Vérification de la condition et mise à jour de date_margin si nécessaire
            if point[2] >= cond:
                date_margin = 1

        # Rétablissement de la valeur originale de date_margin
        date_margin = original_date_margin

        clusters.append(cluster)

    return clusters
def create_cluster_dataframe(clusters):
    cluster_data=[]
    for cluster in clusters:
        consomption_mean = np.mean([point[1] for point in cluster])
        hours_mean = np.mean([point[2] for point in cluster])
        count = len(cluster)
        cluster_data.append([consomption_mean ,hours_mean , count])
    df = pd.DataFrame(cluster_data, columns=['Mean Consomption' , 'Mean Hours', 'Count'])
    return df

# L'appel et l'application de l'algorithme de fusion des nuits sur les données output de fusion des heures pour les nuits.

In [ ]:
Code_list=[]
TalonRef_list=[]
Date1_list=[]
Date2_list=[]
heurev_list=[]
heuref_list=[]
NbrHeure_list=[]
defrence_list=[]
surconso_identifie=[]
Energie=[]
NbrNuit_list=[]
NImpact_conso = []
NperSurconso = []
NPeriode = []



for code in df_NuitOut["Code"].unique():
    df_Nuit = df_NuitOut[df_NuitOut["Code"] == code]
    # Votre code précédent ici...

    # Créer un tableau Numpy à partir des colonnes spécifiées
    X = np.column_stack((df_Nuit.index, df_Nuit['Talon surconso identifie'], df_Nuit['NbrHeures']))

    print(X)
    cond = (23 - df_Nuit['heure fermetur'].iloc[0])+df_Nuit['heure ouverture'].iloc[0]
    marge=df_Nuit['Marge'].iloc[0]
   
    clusters = custom_clustering(X , date_margin=2 , consumption_margin=marge , hours_margin=2 , cond=cond)
    
    df = create_cluster_dataframe(clusters)
    print(df.head(5))

    for (index, row), cluster in zip(df.iterrows(), clusters):
        Energie.append('Electricité')
        heuref_list.append(df_Nuit['heure fermetur'].iloc[0])
        Code_list.append(code)
        TalonRef_list.append(df_Nuit['TalonRef'].iloc[0])
        first_index = int(cluster[0][0])
        Date1_list.append(df_NuitOut['Début surconsommation'].iloc[first_index])
        last_index = int(cluster[-1][0])
        Date2_list.append(df_NuitOut['Fin surconsommation'].iloc[last_index])
        heurev_list.append(df_Nuit['heure ouverture'].iloc[0])
        surconso_identifie.append(arrondir_multiple_de_5(int(row['Mean Consomption'])))
        defrence_list.append(int(row['Mean Consomption']) - df_Nuit['TalonRef'].iloc[0])
        NbrHeure_list.append(round(row['Mean Hours']))
        NbrNuit_list.append(row['Count'])
        surconso = round(row['Mean Hours']) * (int(row['Mean Consomption']) - df_Nuit['TalonRef'].iloc[0]) * row['Count']
        NImpact_conso.append(surconso)

        persurconso = ((int(row['Mean Consomption']) - df_Nuit['TalonRef'].iloc[0]) /  df_Nuit['TalonRef'].iloc[0])*100
        persurconso = int(persurconso)
        persurconso = str(persurconso)
        persurconso = persurconso + '%'
        NperSurconso.append(persurconso)
        NPeriode.append('Nuit')

df_Fusion = pd.DataFrame({'Code':Code_list, 'Energie': Energie,'heure ouverture':heurev_list,'heure fermetur':heuref_list, 'TalonRef':TalonRef_list, 'Début surconsommation':Date1_list, 'Fin surconsommation':Date2_list, 'Talon surconso identifie':surconso_identifie, 'impact':defrence_list,'NbrHeures':NbrHeure_list, 'NbrNuits':NbrNuit_list, 'Impact conso (kWh)':NImpact_conso, '% Surconso':NperSurconso, "Période d'alerte":NPeriode})
grouped_df = df_Fusion.groupby("Code").size().reset_index(name="Nombre de lignes")
grouped_df["Pourcentage de précision"] = np.maximum(30, 100 - (grouped_df["Nombre de lignes"] - 1) * 10)
df = df_Fusion.merge(grouped_df[["Code", "Pourcentage de précision"]], on="Code", how="left")
df['Pourcentage de précision'] = df['Pourcentage de précision'].astype(str) + '%'
df = process_data(df , 5)

# Prétraitement des données et appel du modèle de machine learning (Random Forest) pour nuits.

In [ ]:
data = df[["Talon surconso identifie", "impact", "NbrHeures", "NbrNuits", "Impact conso (kWh)", "% Surconso", "Période d'alerte"]]
data.head()

In [ ]:
print(data.dtypes)

In [ ]:
# Transformation de la colonne '% Surconso' en float
data['% Surconso'] = data['% Surconso'].apply(lambda x: float(x.strip('%')) / 100)

# Affichage du dataframe après la transformation
data.head()

In [ ]:
# Créer un dictionnaire de mapping
mapping = {"Nuit": 0, "Dim": 1, "Jour": 2, "Jour/nuit": 3, "Jour/Nuit": 3}

# Remplacer les valeurs de la colonne 'Catégorie' par les entiers correspondants
data["Période d'alerte"] = data["Période d'alerte"].replace(mapping)

# Afficher les 20 premières lignes du DataFrame
data.head()

In [ ]:
# Faire des prédictions pour chaque ligne de la dataframe
predictions = loaded_model.predict(data)
df["Catégorie"] = predictions
# Créer un dictionnaire de mapping
mapping = {0: "Investigation en cours", 1: "Changement de comportement", 2: "Eclairage", 3: "Froid", 4: "CVC", 5:"Multi-usage"}

# Remplacer les valeurs de la colonne 'Catégorie' par les entiers correspondants
df['Catégorie'] = df['Catégorie'].replace(mapping)
df.head(15)

In [ ]:
df.to_excel(writer, sheet_name='Nuit', index=False)
print('---------------------------Sauvegarde---------------------------')
#writer.save()
writer.close()
print('---------------------------Fin analyse détection auto---------------------------')

In [ ]:
#import pandas as pd

#df_ref = pd.read_excel('fichier2.xlsx', sheet_name='Conso P60', header=0, skiprows=0)
#df_ref.to_csv('data.csv', index=False, encoding='utf-8-sig')